In [20]:
import os
import sys
os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [14]:
# https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [15]:
# Load in STAT 110 for now; convert to lower case
filename = "cleaned_data/STAT/STAT 110.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [18]:
# Map unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# Prepare the dataset of input to output pairs encoded as integers
# TODO: try (padded) sentences instead of sequences of length 100?
# see textblob and https://keras.io/preprocessing/sequence/
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# Reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# Normalize
X = X / float(n_vocab)
# One hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Characters:  349982
Total Vocab:  66
Total Patterns:  349882


In [5]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Define the checkpoint
filepath="keras_checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}-stat110-chars.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [6]:
# Fit
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
349882/349882 [==============================] - 1743s 5ms/step - loss: 2.8459

Epoch 00001: loss improved from inf to 2.84595, saving model to keras_checkpoints/weights-improvement-01-2.8459.hdf5
Epoch 2/20
349882/349882 [==============================] - 1760s 5ms/step - loss: 2.6077

Epoch 00002: loss improved from 2.84595 to 2.60772, saving model to keras_checkpoints/weights-improvement-02-2.6077.hdf5
Epoch 3/20
349882/349882 [==============================] - 1734s 5ms/step - loss: 2.3955

Epoch 00003: loss improved from 2.60772 to 2.39548, saving model to keras_checkpoints/weights-improvement-03-2.3955.hdf5
Epoch 4/20
349882/349882 [==============================] - 1740s 5ms/step - loss: 2.2513

Epoch 00004: loss improved from 2.39548 to 2.25133, saving model to keras_checkpoints/weights-improvement-04-2.2513.hdf5
Epoch 5/20
349882/349882 [==============================] - 1550s 4ms/step - loss: 2.1472

Epoch 00005: loss improved from 2.25133 to 2.14723, saving model 

In [23]:
# load the network weights
filename = "keras_checkpoints/weights-improvement-20-1.6601.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" just make sure you know the cost.
great class to get an introduction to probability theory. problem  "
sets are axasome.  the class is very hirprable to seee the course tiet you den a lot of time and tfe coas and toeerstanding the mrte taan toelrs and toeerstanding the mrte taan toalrstand the material and the class is a great ceasgrgun class that would be iese you ane toeerstanding the material and the class is a great ceasgrgun class that would be iese you ane a lot of time to toeerstand the material. toet 110 is a great class that iave to ae anllitenl metel and the class is a great ceasgrgun class that would be iese you ane toeerstanding the material and the class is a great ceasgrgun class that would be iese you ane a lot of time to toeerstand the material. toet 110 is a great class that iave to ae anllitenl metel and the class is a great ceasgrgun class that would be iese you ane toeerstanding the material and the class is a great ceasgrgun class that would be iese you ane

In [24]:
# Try a bigger model

model2 = Sequential()
model2.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(256))
model2.add(Dropout(0.2))
model2.add(Dense(y.shape[1], activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')

filepath="keras_checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [25]:
# Fit
model2.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
349882/349882 [==============================] - 4968s 14ms/step - loss: 2.6992

Epoch 00001: loss improved from inf to 2.69919, saving model to weights-improvement-01-2.6992-bigger.hdf5
Epoch 2/20
349882/349882 [==============================] - 4715s 13ms/step - loss: 2.1683

Epoch 00002: loss improved from 2.69919 to 2.16834, saving model to weights-improvement-02-2.1683-bigger.hdf5
Epoch 3/20
349882/349882 [==============================] - 4741s 14ms/step - loss: 1.9108

Epoch 00003: loss improved from 2.16834 to 1.91078, saving model to weights-improvement-03-1.9108-bigger.hdf5
Epoch 4/20
349882/349882 [==============================] - 4721s 13ms/step - loss: 1.7746

Epoch 00004: loss improved from 1.91078 to 1.77462, saving model to weights-improvement-04-1.7746-bigger.hdf5
Epoch 5/20
349882/349882 [==============================] - 4713s 13ms/step - loss: 1.6846

Epoch 00005: loss improved from 1.77462 to 1.68462, saving model to weights-improvement-05-1.6846-bigger

In [57]:
# load the network weights
filename = "keras_checkpoints/weights-improvement-19-1.3202-bigger.hdf5"
model2.load_weights(filename)
model2.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model2.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" fe things, such as how often a bus stops at a bus stop. be prepared to study for it, but it's really "
 hard and the course is very difficult. but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould have taken at harvard. the course is very difficult, but it is a great class that i dould ha